In [46]:
import pandas as pd
from scipy import integrate
import datetime

neutron_pvs = ['RAD:Berthold:TotalNeutronRate', 'RAD:ELSE:Neutron', 'RAD:Thermo1:Neutron',\
    'RAD:Thermo2:Neutron', 'RAD:Thermo3:Neutron', 'RAD:Thermo4:Neutron', 'RAD:Thermo5:Neutron',\
        'RAD:Thermo6:Neutron', 'RAD:Thermo7:Neutron', 'RAD:Thermo8:Neutron', 'RAD:Thermo9:Neutron']

FROM_NOW = False
if FROM_NOW:
    final_date = datetime.date.today()
    initial_date = datetime.date.today() - datetime.timedelta(3)
else:
    initial_date = '2022-01-08'
    initial_time = '00:00:00'
    final_date = '2022-01-09'
    final_time = '00:00:00'


In [51]:
# Recupera dados do archiver
dose_rate = []
url_prefix = 'http://10.0.38.42/retrieval/data/getData.csv?pv='
url_posfix = '&from={}T{}00.000Z&to={}T{}00.000Z'.format(initial_date, initial_time, final_date, final_time)
neutron_dfs = {}

for _ in neutron_pvs:
    temp = pd.read_csv('{}{}{}'.format(url_prefix, _, url_posfix),\
         header = None, names = ["secs", "Dose Rate (uSv/h)", "severity", "status", "nanos"])
    temp['date'] = time = pd.to_datetime(temp['secs'], unit='s')
    temp['time (h)'] = (temp['secs'].subtract(temp['secs'][0]))/3600
    temp.drop('nanos', inplace=True, axis=1)
    temp.drop('severity', inplace=True, axis=1)
    temp.drop('status', inplace=True, axis=1)
    temp.drop('secs', inplace=True, axis=1)
    neutron_dfs[_] = temp

HTTPError: HTTP Error 400: 

In [ ]:
dose = {}
for _ in neutron_pvs:
    dose[_] = integrate.simps(neutron_dfs[_]['Dose Rate (uSv/h)'])
print(dose.values())

In [48]:
print(neutron_dfs['RAD:Berthold:TotalNeutronRate'])

      Dose Rate (uSv/h)                date   time (h)
0              0.014877 2022-01-07 23:59:57   0.000000
1              0.022317 2022-01-08 00:00:07   0.002778
2              0.029757 2022-01-08 00:00:27   0.008333
3              0.037201 2022-01-08 00:00:57   0.016667
4              0.029762 2022-01-08 00:01:04   0.018611
...                 ...                 ...        ...
2129           0.022320 2022-01-08 23:57:58  23.966944
2130           0.014880 2022-01-08 23:58:02  23.968056
2131           0.007440 2022-01-08 23:59:34  23.993611
2132           0.014878 2022-01-08 23:59:43  23.996111
2133           0.007439 2022-01-08 23:59:59  24.000556

[2134 rows x 3 columns]
